# Chatbot
- [reference] https://python.langchain.com/v0.2/docs/tutorials/chatbot/

## Quickstart

In [ ]:
%%script echo skipping
import getpass # 사용자에게 비밀번호나 API 키와 같은 민감한 정보를 입력받을 때 사용되는 파이썬 표준 라이브러리입니다.
import os # 운영체제와 상호작용하는 기능을 제공하는 파이썬 표준 라이브러리입니다. 환경 변수 설정 등에 사용됩니다.

# OpenAI API 키를 환경 변수로 설정합니다.
# getpass.getpass() 함수는 사용자에게 콘솔에서 입력 프롬프트를 표시하고, 입력된 내용을 에코(표시)하지 않으므로 보안에 유리합니다.
# 이렇게 설정된 환경 변수는 LangChain이 OpenAI 모델과 통신할 때 자동으로 사용됩니다.
os.environ["OPENAI_API_KEY"] = getpass.getpass()

# LangChain의 OpenAI Chat 모델을 임포트합니다.
# 이 클래스는 OpenAI의 채팅 기반 모델(예: GPT-3.5 Turbo, GPT-4)과 상호작용하기 위한 인터페이스를 제공합니다.
from langchain_openai import ChatOpenAI

# ChatOpenAI 인스턴스를 생성합니다.
# model="gpt-3.5-turbo"는 사용할 OpenAI 모델의 이름을 지정합니다. 여기서는 GPT-3.5 Turbo 모델을 사용합니다.
model = ChatOpenAI(model="gpt-3.5-turbo")

# --- 모델을 직접 사용하는 방법 (상태 비저장 방식) ---
# "ChatModels are instances of LangChain 'Runnables', which means they expose a standard interface for interacting with them."
# LangChain의 Chat 모델은 "Runnable" 인터페이스를 구현합니다. 이는 .invoke(), .batch(), .stream()과 같은 표준 메서드를 통해 상호작용할 수 있음을 의미합니다.

# 단순히 모델을 호출하려면 .invoke 메서드에 메시지 리스트를 전달합니다.
# LangChain의 핵심 메시지 유형 중 하나인 HumanMessage를 임포트합니다.
# HumanMessage는 사용자가 보낸 메시지를 나타냅니다.
from langchain_core.messages import HumanMessage

# 모델에 직접 첫 번째 질문을 던집니다.
# .invoke() 메서드는 모델을 실행하고 응답을 반환합니다.
# 이 경우, "Hi! I'm Bob"이라는 단일 HumanMessage를 포함하는 리스트를 전달합니다.
response1 = model.invoke([HumanMessage(content="Hi! I'm Bob")])

# 모델의 응답을 출력합니다.
# AIMessage는 AI(모델)가 보낸 메시지를 나타냅니다.
# content: 모델의 실제 텍스트 응답.
# response_metadata: API 호출에 대한 추가 메타데이터 (토큰 사용량, 모델 이름 등).
# id: 해당 호출의 고유 ID (LangSmith 트레이싱 등에 사용).
# usage_metadata: 토큰 사용량에 대한 간략한 정보.
print(response1)
# 출력 예시:
# AIMessage(content='Hello Bob! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 12, 'total_tokens': 22}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d939617f-0c3b-45e9-a93f-13dafecbd4b5-0', usage_metadata={'input_tokens': 12, 'output_tokens': 10, 'total_tokens': 22})


# "The model on its own does not have any concept of state."
# 모델 자체는 "상태(state)" 개념을 가지고 있지 않습니다. 즉, 이전 대화 내용을 자동으로 기억하지 못합니다.
# 이전에 "Hi! I'm Bob"이라고 말했지만, 모델은 그 정보를 저장하지 않습니다.

# "For example, if you ask a followup question:"
# 예를 들어, 후속 질문을 하는 경우:
# 모델에 두 번째 질문을 던집니다. 이전 대화 기록은 전달하지 않습니다.
response2 = model.invoke([HumanMessage(content="What's my name?")])

# 모델의 응답을 출력합니다.
# 모델은 이전에 "Bob"이라는 정보를 받았음에도 불구하고, 해당 정보를 기억하지 못하고 이름을 모른다고 답변합니다.
print(response2)
# 출력 예시:
# AIMessage(content="I'm sorry, I don't have access to personal information unless you provide it to me. How may I assist you today?", response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 12, 'total_tokens': 38}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-47bc8c20-af7b-4fd2-9345-f0e9fdf18ce3-0', usage_metadata={'input_tokens': 12, 'output_tokens': 26, 'total_tokens': 38})


# "Let's take a look at the example LangSmith trace" (LangSmith 트레이스는 코드 실행 환경에서 LangSmith를 사용하고 있을 때 볼 수 있습니다.)
# "We can see that it doesn't take the previous conversation turn into context, and cannot answer the question. This makes for a terrible chatbot experience!"
# 위 결과는 모델이 이전 대화 턴(turn)을 컨텍스트로 사용하지 않았고, 따라서 질문에 답할 수 없음을 보여줍니다. 이는 끔찍한 챗봇 경험을 만듭니다!

# "To get around this, we need to pass the entire conversation history into the model."
# 이 문제를 해결하려면 전체 대화 기록을 모델에 전달해야 합니다.

# "Let's see what happens when we do that:"
# 그렇게 했을 때 어떤 일이 발생하는지 봅시다:
# LangChain의 AIMessage를 임포트합니다. 이는 AI(모델)가 보낸 메시지를 나타냅니다.
from langchain_core.messages import AIMessage

# 모델에 전체 대화 기록을 포함하여 질문을 던집니다.
# 첫 번째 질문("Hi! I'm Bob"), 그에 대한 모델의 응답("Hello Bob! How can I assist you today?"),
# 그리고 현재 질문("What's my name?")까지 모든 메시지를 리스트로 묶어 전달합니다.
response3 = model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"), # 첫 번째 사용자 메시지
        AIMessage(content="Hello Bob! How can I assist you today?"), # 첫 번째 모델 응답 (가정 또는 이전 호출 결과)
        HumanMessage(content="What's my name?"), # 두 번째 사용자 질문
    ]
)

# 모델의 응답을 출력합니다.
# 이제 모델은 이전 대화 기록을 통해 "Bob"이라는 정보를 파악하고 올바르게 답변합니다.
print(response3)
# 출력 예시:
# AIMessage(content='Your name is Bob. How can I help you, Bob?', response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 35, 'total_tokens': 48}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9f90291b-4df9-41dc-9ecf-1ee1081f4490-0', usage_metadata={'input_tokens': 35, 'output_tokens': 13, 'total_tokens': 48})


# "And now we can see that we get a good response!"
# 그리고 이제 좋은 응답을 얻을 수 있음을 확인할 수 있습니다!

# "This is the basic idea underpinning a chatbot's ability to interact conversationally. So how do we best implement this?"
# 이것이 챗봇이 대화식으로 상호작용할 수 있는 능력의 기본 아이디어입니다. 그렇다면 이를 어떻게 가장 잘 구현할 수 있을까요?
# 이 질문은 다음 단계로 넘어가 LangChain의 Memory 또는 ConversationChain과 같은 기능을 사용하여 대화 기록을 자동으로 관리하는 방법을 설명할 것임을 암시합니다.

In [103]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv

load_dotenv()

model = ChatOpenAI()

ai_message = model.invoke([HumanMessage(content='Hi, I`m Bob')])

ai_message.content

'Hello Bob, how can I assist you today?'

In [104]:
ai_message_02 = model.invoke('what`s my name?')

ai_message_02.content

"I'm sorry, I don't have that information. Can you please tell me your name?"

In [105]:
ai_message_03 = model.invoke([HumanMessage(content='what`s my name?')])

ai_message_03.content

"I'm sorry, I don't know your name as I am an AI assistant and do not have access to personal information. Could you please tell me your name?"

In [106]:
from langchain_core.messages import AIMessage

ai_message_04 = model.invoke(
    [
        HumanMessage(content='Hi I`m Bob.'),
        AIMessage(content='Hello Bob! How can I assist you today?'),
        HumanMessage(content='What`s my name?')
    ]
)

ai_message_04.content

'Your name is Bob.'

## Message History

In [ ]:
%%script echo skipping
# 'getpass'와 'os' 모듈은 이전 설명에서 이미 다루었으므로 여기서는 생략합니다.
import getpass
import os

# OpenAI API 키를 환경 변수로 설정합니다. 보안을 위해 'getpass'를 사용하여 입력값을 숨깁니다.
os.environ["OPENAI_API_KEY"] = getpass.getpass()

# LangChain의 ChatOpenAI 모델을 임포트합니다.
from langchain_openai import ChatOpenAI

# GPT-3.5 Turbo 모델 인스턴스를 생성합니다.
model = ChatOpenAI(model="gpt-3.5-turbo")

# --- 메시지 기록을 사용하여 모델에 상태를 부여하는 방법 ---

# "We can use a Message History class to wrap our model and make it stateful."
# 모델을 감싸고 상태를 가지도록 만들기 위해 'Message History' 클래스를 사용할 수 있습니다.
# "This will keep track of inputs and outputs of the model, and store them in some datastore."
# 이는 모델의 입력과 출력을 추적하고, 일부 데이터 저장소에 저장합니다.
# "Future interactions will then load those messages and pass them into the chain as part of the input."
# 이후의 상호작용은 해당 메시지들을 로드하여 입력의 일부로 체인에 전달할 것입니다.

# "First, let's make sure to install langchain-community, as we will be using an integration in there to store message history."
# 메시지 기록을 저장하는 데 사용될 통합 기능을 위해 먼저 'langchain-community'를 설치해야 합니다.
# 이 줄은 Jupyter Notebook이나 IPython 환경에서 셀(cell) 명령어로 패키지를 설치하는 것을 나타냅니다.
# 실제 파이썬 스크립트에서는 'pip install langchain_community'를 터미널에서 실행해야 합니다.
# %pip install langchain_community

# "After that, we can import the relevant classes and set up our chain which wraps the model and adds in this message history."
# 그 후, 관련 클래스를 임포트하고 모델을 감싸고 메시지 기록을 추가하는 체인을 설정할 수 있습니다.

# LangChain의 핵심 메시지 기록 관련 클래스들을 임포트합니다.
# BaseChatMessageHistory: 모든 메시지 기록 저장소의 기본 추상 클래스입니다.
# InMemoryChatMessageHistory: 메시지 기록을 메모리에 저장하는 간단한 구현체입니다. (휘발성)
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)

# RunnableWithMessageHistory: 메시지 기록 기능을 LangChain Runnable에 추가하는 클래스입니다.
# 이를 사용하면 체인에 대화 기록 관리 기능을 쉽게 통합할 수 있습니다.
from langchain_core.runnables.history import RunnableWithMessageHistory

# 세션 ID별 메시지 기록을 저장할 간단한 인메모리 저장소(Python 딕셔너리)입니다.
# 실제 애플리케이션에서는 데이터베이스(Redis, DynamoDB 등)를 사용합니다.
store = {}

# 'get_session_history' 함수를 정의합니다.
# "A key part here is the function we pass into as the get_session_history."
# 여기서 핵심 부분은 'get_session_history'로 전달하는 함수입니다.
# "This function is expected to take in a session_id and return a Message History object."
# 이 함수는 'session_id'를 입력받아 Message History 객체를 반환해야 합니다.
# "This session_id is used to distinguish between separate conversations, and should be passed in as part of the config when calling the new chain (we'll show how to do that)."
# 이 'session_id'는 별개의 대화를 구별하는 데 사용되며, 새 체인을 호출할 때 'config'의 일부로 전달되어야 합니다.
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    # 주어진 session_id가 store 딕셔너리에 없으면, 새로운 InMemoryChatMessageHistory 객체를 생성하여 저장합니다.
    # 이는 새 대화가 시작될 때 해당 세션에 대한 기록 저장소를 초기화합니다.
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    # 해당 session_id에 연결된 메시지 기록 객체를 반환합니다.
    return store[session_id]

# RunnableWithMessageHistory 인스턴스를 생성합니다.
# 첫 번째 인자: 메시지 기록이 적용될 기본 Runnable (여기서는 ChatOpenAI 모델).
# 두 번째 인자: 세션 ID를 받아 메시지 기록 객체를 반환하는 함수 (위에서 정의한 get_session_history).
# 이 'with_message_history'는 이제 상태를 관리하는 새로운 Runnable이 됩니다.
with_message_history = RunnableWithMessageHistory(model, get_session_history)

# API Reference: BaseChatMessageHistory | InMemoryChatMessageHistory | RunnableWithMessageHistory
# 해당 클래스들에 대한 LangChain 공식 문서 참조 링크를 제공합니다.

# "We now need to create a config that we pass into the runnable every time."
# 이제 매번 Runnable에 전달할 'config'를 생성해야 합니다.
# "This config contains information that is not part of the input directly, but is still useful."
# 이 'config'는 직접적인 입력의 일부는 아니지만 여전히 유용한 정보를 포함합니다.
# "In this case, we want to include a session_id."
# 이 경우, 'session_id'를 포함하고자 합니다.

# 대화 세션을 식별하는 데 사용될 'config' 딕셔너리를 정의합니다.
# 'configurable' 키 아래에 'session_id'를 설정합니다. 'abc2'는 특정 대화 세션을 나타내는 ID입니다.
config = {"configurable": {"session_id": "abc2"}}

# 메시지 기록이 적용된 체인('with_message_history')을 호출합니다.
# 첫 번째 인자: 현재 사용자 메시지 리스트.
# 'config' 키워드 인자를 사용하여 위에 정의한 'config' 딕셔너리를 전달합니다.
# 이 'config'를 통해 'get_session_history' 함수가 'abc2' 세션에 대한 기록을 가져오거나 생성하게 됩니다.
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bob")],
    config=config,
)

# 모델의 응답 내용을 출력합니다.
# 이 응답은 'abc2' 세션의 메시지 기록에 자동으로 저장됩니다.
print(response.content)
# 출력: 'Hi Bob! How can I assist you today?'

# "Great! Our chatbot now remembers things about us."
# 좋습니다! 이제 우리 챗봇은 우리에 대해 기억합니다.

# 'abc2' 세션에 대해 후속 질문을 합니다.
# 이 호출은 'abc2' 세션의 기존 메시지 기록을 자동으로 로드하고, 새로운 질문을 추가하여 모델에 전달합니다.
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

# 모델의 응답 내용을 출력합니다.
# 모델은 이전 "Hi! I'm Bob" 메시지를 기억하고 이름을 올바르게 답변합니다.
print(response.content)
# 출력: 'Your name is Bob. How can I help you today, Bob?'

# "If we change the config to reference a different session_id, we can see that it starts the conversation fresh."
# 만약 'config'를 다른 'session_id'를 참조하도록 변경하면, 대화가 새로 시작되는 것을 볼 수 있습니다.

# 새로운 세션 ID 'abc3'을 가진 'config'를 생성합니다.
config = {"configurable": {"session_id": "abc3"}}

# 새로운 세션('abc3')으로 'What's my name?' 질문을 합니다.
# 'abc3' 세션에는 이전 대화 기록이 없으므로 모델은 이름을 알지 못한다고 답변합니다.
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

print(response.content)
# 출력: "I'm sorry, I cannot determine your name as I am an AI assistant and do not have access to that information."

# "However, we can always go back to the original conversation (since we are persisting it in a database)"
# 하지만, 우리는 언제든지 원래 대화로 돌아갈 수 있습니다 (데이터베이스에 저장하고 있으므로).
# 여기서 "데이터베이스"는 이 예시에서는 'store' 딕셔너리를 의미합니다. 실제 애플리케이션에서는 영구적인 DB를 사용합니다.

# 다시 원래 세션 ID 'abc2'로 돌아갑니다.
config = {"configurable": {"session_id": "abc2"}}

# 'abc2' 세션으로 'What's my name?' 질문을 합니다.
# 'abc2' 세션의 기록이 다시 로드되어 모델은 "Bob"이라는 이름을 기억합니다.
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

print(response.content)
# 출력: 'Your name is Bob. How can I assist you today, Bob?'

# "This is how we can support a chatbot having conversations with many users!"
# 이것이 챗봇이 여러 사용자와 대화하는 것을 지원하는 방법입니다!
# 각 사용자(또는 각 대화)에 고유한 'session_id'를 할당함으로써 독립적인 대화 흐름을 유지할 수 있습니다.

# "Right now, all we've done is add a simple persistence layer around the model. We can start to make the more complicated and personalized by adding in a prompt template."
# 현재까지는 모델 주위에 간단한 지속성 계층(메시지 기록 저장)을 추가한 것이 전부입니다.
# 프롬프트 템플릿을 추가하여 이를 더 복잡하고 개인화된 기능으로 만들 수 있습니다.
# 이는 다음 단계에서 'prompt'와 'model'을 결합하여 더 정교한 챗봇을 만드는 방법을 보여줄 것임을 암시합니다.

In [107]:
from langchain_core.chat_history import(
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [108]:
config = {'configurable': {'session_id': 'abc2'}}

In [109]:
response = with_message_history.invoke(
    [HumanMessage(content='Hi I`m Bob')],
    config=config,
)

response.content

'Hello Bob! How can I assist you today?'

In [110]:
response = with_message_history.invoke(
    [HumanMessage(content='What`s my name?')],
    config=config
)

response.content

'Your name is Bob.'

In [111]:
config = {'configurable': {'session_id': 'abc3'}}

response = with_message_history.invoke(
    [HumanMessage(content='what`s my name?')],
    config=config,
)

response.content

"I'm sorry, I cannot determine your name as I am an AI assistant and do not have access to personal information."

In [112]:
config = {'configurable': {'session_id': 'abc2'}}

response = with_message_history.invoke(
    [HumanMessage(content='what`s my name?')],
    config=config,
)

response.content

'Your name is Bob.'

In [113]:
get_session_history('abc2').messages

[HumanMessage(content='Hi I`m Bob', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hello Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 12, 'total_tokens': 22, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BcQsnwkscN8UQJDZQwpUM7CTi3XzI', 'finish_reason': 'stop', 'logprobs': None}, id='run--0ee5a600-a65a-451f-97bf-eb637826efe0-0', usage_metadata={'input_tokens': 12, 'output_tokens': 10, 'total_tokens': 22, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 HumanMessage(content='What`s my name?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Your name is Bo

In [114]:
## 첫 번째 질문 : HumanMessage 출력
type(get_session_history('abc2').messages)

list

In [115]:
get_session_history('abc2').messages[0].content

'Hi I`m Bob'

In [116]:
HumanMessage_1 = [msg for msg in get_session_history('abc2').messages if isinstance(msg, HumanMessage)]
for msg in HumanMessage_1:
    print(f'HumanMessage: {msg.content}')
print('\n')

HumanMessage: Hi I`m Bob
HumanMessage: What`s my name?
HumanMessage: what`s my name?




In [117]:
for message in get_session_history('abc2').messages:
    print(message.content)
    print('*'*50)

Hi I`m Bob
**************************************************
Hello Bob! How can I assist you today?
**************************************************
What`s my name?
**************************************************
Your name is Bob.
**************************************************
what`s my name?
**************************************************
Your name is Bob.
**************************************************


## Prompt Templates

In [ ]:
%%script echo skipping
# 필요한 모듈들을 임포트합니다.
# getpass: 사용자에게 비밀번호/API 키를 안전하게 입력받기 위함.
# os: 환경 변수를 설정하기 위함.
import getpass
import os

# LangChain의 OpenAI Chat 모델을 임포트합니다.
from langchain_openai import ChatOpenAI

# LangChain의 핵심 메시지 기록 관련 클래스들을 임포트합니다.
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
# 메시지 기록 기능을 Runnable에 추가하는 클래스입니다.
from langchain_core.runnables.history import RunnableWithMessageHistory
# LangChain의 프롬프트 템플릿 관련 클래스들을 임포트합니다.
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
# HumanMessage: 사용자의 메시지를 나타내는 클래스.
from langchain_core.messages import HumanMessage

# --- 기본 설정 (이전 섹션에서 반복된 부분) ---
# OpenAI API 키를 환경 변수로 설정합니다.
os.environ["OPENAI_API_KEY"] = getpass.getpass()

# ChatOpenAI 모델 인스턴스를 생성합니다.
model = ChatOpenAI(model="gpt-3.5-turbo")

# 세션 ID별 메시지 기록을 저장할 인메모리 딕셔너리입니다.
store = {}

# 세션 ID를 받아 메시지 기록 객체를 반환하는 함수입니다.
# 해당 세션 ID의 기록이 없으면 새로 생성하고, 있으면 기존 기록을 반환합니다.
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# --- 프롬프트 템플릿을 사용하여 모델에 시스템 지시 추가하기 ---

# "Prompt Templates help to turn raw user information into a format that the LLM can work with."
# 프롬프트 템플릿은 원시 사용자 정보를 LLM(대규모 언어 모델)이 작업할 수 있는 형식으로 변환하는 데 도움을 줍니다.
# "In this case, the raw user input is just a message, which we are passing to the LLM."
# 이 경우, 원시 사용자 입력은 단순히 메시지이며, 이를 LLM에 전달하고 있습니다.
# "Let's now make that a bit more complicated."
# 이제 이를 조금 더 복잡하게 만들어 봅시다.
# "First, let's add in a system message with some custom instructions (but still taking messages as input)."
# 먼저, 일부 사용자 지정 지시(여전히 메시지를 입력으로 받음)가 포함된 시스템 메시지를 추가해 봅시다.
# "Next, we'll add in more input besides just the messages."
# 다음으로, 메시지 외에 더 많은 입력을 추가할 것입니다.

# "First, let's add in a system message. To do this, we will create a ChatPromptTemplate."
# 먼저 시스템 메시지를 추가해 봅시다. 이를 위해 ChatPromptTemplate을 생성할 것입니다.
# "We will utilize MessagesPlaceholder to pass all the messages in."
# 모든 메시지를 전달하기 위해 MessagesPlaceholder를 활용할 것입니다.

# ChatPromptTemplate: 채팅 메시지 형식을 가진 프롬프트 템플릿을 생성합니다.
# from_messages(): 메시지 튜플(역할, 내용) 리스트를 받아 템플릿을 생성합니다.
# ("system", "..."): 시스템 메시지를 정의합니다. 이는 모델의 전반적인 행동을 지시합니다.
# MessagesPlaceholder(variable_name="messages"): 'messages'라는 변수 이름으로 들어오는 메시지 리스트를 여기에 삽입하라는 지시자입니다.
# 이는 이전 대화 기록이나 현재 사용자 질문이 이 자리에 동적으로 채워지도록 합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"), # 여기에 실제 대화 메시지가 삽입됩니다.
    ]
)

# 프롬프트 템플릿과 모델을 파이프(|)로 연결하여 새로운 'chain'을 만듭니다.
# 'prompt'의 출력(정의된 시스템 메시지와 입력된 메시지 리스트로 구성된 메시지 리스트)이 'model'의 입력으로 전달됩니다.
chain = prompt | model

# API Reference: ChatPromptTemplate | MessagesPlaceholder
# 해당 클래스들에 대한 LangChain 공식 문서 참조 링크를 제공합니다.

# "Note that this slightly changes the input type - rather than pass in a list of messages, we are now passing in a dictionary with a messages key where that contains a list of messages."
# 여기서 입력 유형이 약간 변경됩니다. 메시지 리스트를 직접 전달하는 대신, 이제 메시지 리스트를 포함하는 'messages' 키를 가진 딕셔너리를 전달합니다.
# 이는 'prompt'가 단순히 메시지 리스트가 아닌, 명명된 변수(여기서는 'messages')를 포함하는 딕셔너리를 기대하기 때문입니다.

# 새로 정의된 'chain'을 호출합니다.
# 입력은 'messages' 키를 가진 딕셔너리 형태입니다.
response = chain.invoke({"messages": [HumanMessage(content="hi! I'm bob")]})

# 모델의 응답 내용을 출력합니다.
# 시스템 메시지("You are a helpful assistant...")의 지시가 적용되어 모델이 "Bob"에게 인사합니다.
print(response.content)
# 예상 출력: 'Hello Bob! How can I assist you today?'

# --- 프롬프트 템플릿이 적용된 체인에 메시지 기록 추가하기 ---

# "We can now wrap this in the same Messages History object as before"
# 이제 이 체인을 이전과 동일한 메시지 기록 객체로 감쌀 수 있습니다.
# 'RunnableWithMessageHistory'는 이전에 정의한 'chain'(프롬프트 + 모델)과 'get_session_history' 함수를 사용하여 새로운 Runnable을 생성합니다.
# 이 'with_message_history'는 이제 시스템 메시지가 적용된 상태를 관리하는 체인이 됩니다.
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

# 새로운 세션 ID 'abc5'를 가진 'config'를 정의합니다.
config = {"configurable": {"session_id": "abc5"}}

# 새로운 세션으로 첫 번째 질문을 합니다.
# 입력은 여전히 'messages' 키를 가진 딕셔너리 형태입니다.
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="Hi! I'm Jim")]}, # 'messages' 키로 메시지 리스트 전달
    config=config, # 'abc5' 세션으로 지정
)

print(response.content)
# 출력: 'Hello, Jim! How can I assist you today?' (시스템 지시에 따라 응답)

# 'abc5' 세션으로 후속 질문을 합니다.
# 'with_message_history'가 'abc5' 세션의 기록을 자동으로 로드하여 모델에 전달하므로, 모델은 'Jim'이라는 이름을 기억합니다.
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="What's my name?")]},
    config=config,
)

print(response.content)
# 출력: 'Your name is Jim.'

# --- 프롬프트 템플릿에 추가적인 입력 변수 (`language`) 추가하기 ---

# "Awesome! Let's now make our prompt a little bit more complicated."
# 좋습니다! 이제 프롬프트를 조금 더 복잡하게 만들어 봅시다.
# "Let's assume that the prompt template now looks something like this:"
# 프롬프트 템플릿이 이제 다음과 같이 생겼다고 가정해 봅시다:

# 'language'라는 새로운 입력 변수를 포함하는 ChatPromptTemplate을 정의합니다.
# 시스템 메시지에 "{language}" 플레이스홀더가 추가되어, 모델이 특정 언어로 응답하도록 지시할 수 있습니다.
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.", # 새로운 변수 '{language}' 추가
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

# 새로운 프롬프트 템플릿과 모델을 연결하여 'chain'을 다시 정의합니다.
chain = prompt | model

# "Note that we have added a new language input to the prompt. We can now invoke the chain and pass in a language of our choice."
# 프롬프트에 새로운 'language' 입력을 추가했습니다. 이제 체인을 호출하고 원하는 언어를 전달할 수 있습니다.

# 'chain'을 호출할 때 'language' 변수를 함께 전달합니다.
# 'messages'와 'language' 두 가지 키를 포함하는 딕셔너리를 입력으로 사용합니다.
response = chain.invoke(
    {"messages": [HumanMessage(content="hi! I'm bob")], "language": "Spanish"} # 'language' 변수 추가
)

print(response.content)
# 출력: '¡Hola, Bob! ¿En qué puedo ayudarte hoy?' (스페인어로 응답)

# --- 다중 입력 변수를 가진 체인에 메시지 기록 추가하기 (input_messages_key 지정) ---

# "Let's now wrap this more complicated chain in a Message History class."
# 이제 이 더 복잡한 체인을 메시지 기록 클래스로 감싸 봅시다.
# "This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history."
# 이번에는 입력에 여러 키가 있으므로, 채팅 기록을 저장하는 데 사용할 올바른 키를 지정해야 합니다.
# 'chain'의 입력은 이제 'messages'와 'language'를 포함하는 딕셔너리입니다.
# 'RunnableWithMessageHistory'는 어떤 키가 실제 대화 메시지 리스트인지 알아야 기록을 올바르게 저장하고 로드할 수 있습니다.

# 'RunnableWithMessageHistory'를 생성할 때 'input_messages_key="messages"'를 명시적으로 지정합니다.
# 이는 'chain'에 전달되는 딕셔너리에서 'messages'라는 키가 실제 대화 기록을 담고 있음을 'RunnableWithMessageHistory'에 알려줍니다.
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages", # 대화 기록을 저장/로드할 입력 키를 명시적으로 지정
)

# 새로운 세션 ID 'abc11'을 가진 'config'를 정의합니다.
config = {"configurable": {"session_id": "abc11"}}

# 'abc11' 세션으로 첫 번째 질문을 합니다.
# 'messages'와 'language'를 모두 포함하는 딕셔너리를 입력으로 전달합니다.
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm todd")], "language": "Spanish"},
    config=config,
)

print(response.content)
# 출력: '¡Hola Todd! ¿En qué puedo ayudarte hoy?' (스페인어로 응답하며 메시지 기록에 저장됨)

# 'abc11' 세션으로 후속 질문을 합니다.
# 'input_messages_key="messages"' 덕분에 'with_message_history'는 'messages' 키 아래의 대화 기록을 올바르게 로드하고,
# 'language'는 현재 호출의 입력으로 계속 전달됩니다.
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Spanish"},
    config=config,
)

print(response.content)
# 출력: 'Tu nombre es Todd.' (이름을 기억하고 스페인어로 응답)

# "To help you understand what's happening internally, check out this LangSmith trace"
# 내부적으로 어떤 일이 일어나고 있는지 이해하는 데 도움이 되도록 이 LangSmith 추적을 확인하세요.
# LangSmith는 LangChain 애플리케이션의 실행 흐름을 시각화하고 디버깅하는 데 유용한 도구입니다.

In [118]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [('system', 'You are a helpful assistant. Answer all questinos to the best of your ability.'),
    MessagesPlaceholder(variable_name='messages'),])

chain = prompt | model

In [119]:
response = chain.invoke({'messages':[HumanMessage(content='Hi, I`m Bob.')]})

response.content

'Hello Bob! How can I assist you today?'

In [120]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)
config = {'configurable': {'session_id': 'abc5'}}
response = with_message_history.invoke(
    [HumanMessage(content='Hi I`m Jim')],
    config=config
)

response.content

'Hello Jim! How can I assist you today?'

In [121]:
response = with_message_history.invoke(
    [HumanMessage(content='What`s my name?')],
    config=config
)

response.content

'Your name is Jim.'

In [122]:
for message in get_session_history('abc5').messages:
    print(f'[{message.type.upper()}]: {message.content}')

[HUMAN]: Hi I`m Jim
[AI]: Hello Jim! How can I assist you today?
[HUMAN]: What`s my name?
[AI]: Your name is Jim.


In [123]:
for message in get_session_history('abc5').messages:
    if message.type == 'human':
        print(f'[{message.type.upper()}]: {message.content}')

[HUMAN]: Hi I`m Jim
[HUMAN]: What`s my name?


In [124]:
prompt = ChatPromptTemplate.from_messages(
    [('system', 'You are a helpful assistant. Answer all questions to the best of your ability in {language}.',),
     MessagesPlaceholder(variable_name='messages')],
)

chain = prompt | model

In [125]:
response = chain.invoke(
    {'messages': [HumanMessage(content='hi! I`m bob')], 'language': 'Spanish'}
)

response.content

'¡Hola, Bob! ¿En qué puedo ayudarte hoy?'

In [126]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key='messages'
)

config = {'configurable': {'session_id': 'abc11'}}

response = with_message_history.invoke(
    {'messages': [HumanMessage(content='hi! I`m todd')], 'language': 'Spanish'},
    config=config,
)

response.content

'¡Hola Todd! ¿En qué puedo ayudarte hoy?'

In [127]:
response = with_message_history.invoke(
    {'messages': [HumanMessage(content='what`s my name?')], 'language': 'Spanish'},
    config=config
)

response.content

'Tu nombre es Todd.'

## Managing Conversation History

In [128]:
%%script echo skipping
# 챗봇을 만들 때 반드시 이해해야 할 중요한 개념 중 하나는 '대화 기록'을 어떻게 관리하는지입니다.
# 대화 기록을 제대로 관리하지 않으면, 메시지 목록이 끝없이 늘어나게 되고,
# 결국 LLM(대규모 언어 모델)의 '컨텍스트 창(Context Window)'을 넘쳐버릴 수 있습니다.
# 컨텍스트 창은 LLM이 한 번에 처리할 수 있는 정보의 양을 의미합니다.
# 이 창을 넘어서면 LLM은 이전 대화를 기억하지 못하거나, 심지어 오류를 발생시킬 수 있습니다.
# 따라서 LLM에 전달하는 메시지 크기를 제한하는 단계를 추가하는 것이 매우 중요합니다.

# 중요한 점은, 이 '메시지 크기 제한' 작업은 '프롬프트 템플릿(Prompt Template)' 이전에 수행해야 하고,
# '메시지 기록(Message History)'에서 이전 메시지를 불러온 '이후'에 수행해야 한다는 것입니다.
# 즉, 이전 대화를 가져온 다음, 그 대화 목록을 줄이고, 그 줄여진 대화를 프롬프트로 전달해야 합니다.

# 우리는 'prompt' 앞에 간단한 단계를 추가하여 'messages' 키를 적절하게 수정함으로써 이를 수행할 수 있습니다.
# 그리고 이 새로운 체인(chain)을 'Message History' 클래스로 감쌀 것입니다.

# LangChain은 메시지 목록을 관리하기 위한 몇 가지 내장된 헬퍼(helper) 기능을 제공합니다.
# 이 경우, 우리는 'trim_messages' 헬퍼를 사용하여 모델에 보내는 메시지 수를 줄일 것입니다.
# trim_messages는 우리가 유지하고자 하는 토큰(token) 수를 지정할 수 있게 해주며,
# 시스템 메시지를 항상 포함할지, 부분 메시지를 허용할지 등 다른 매개변수(parameter)도 설정할 수 있습니다.

from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, trim_messages

# trim_messages 함수를 사용하여 메시지 트리머(trimmer)를 생성합니다.
trimmer = trim_messages(
    max_tokens=65,  # 유지할 최대 토큰 수입니다. 텍스트를 구성하는 최소 단위를 '토큰'이라고 생각하면 됩니다.
    strategy="last", # 메시지를 자를 전략입니다. "last"는 가장 최근 메시지부터 유지하고, 오래된 메시지를 제거합니다.
    token_counter="model", # 토큰 수를 계산하는 데 사용할 함수 또는 모델입니다. 여기서는 모델이 알아서 토큰 수를 계산하도록 합니다.
    include_system=True, # 시스템 메시지(SystemMessage)를 항상 포함할지 여부입니다. 챗봇의 기본 지시사항이므로 보통 True로 설정합니다.
    allow_partial=False, # 부분 메시지를 허용할지 여부입니다. False면 메시지 중간에 잘리지 않습니다.
    start_on="human", # 어떤 메시지부터 카운트를 시작할지 지정합니다. "human"은 사용자 메시지부터 시작합니다.
)

# 예시 대화 기록입니다.
messages = [
    SystemMessage(content="you're a good assistant"), # 시스템 메시지: 챗봇의 역할 또는 지시사항
    HumanMessage(content="hi! I'm bob"), # 사용자 메시지: "안녕! 나는 밥이야"
    AIMessage(content="hi!"), # AI 메시지: "안녕!"
    HumanMessage(content="I like vanilla ice cream"), # 사용자 메시지: "나는 바닐라 아이스크림을 좋아해"
    AIMessage(content="nice"), # AI 메시지: "좋네"
    HumanMessage(content="whats 2 + 2"), # 사용자 메시지: "2 더하기 2는 뭐야"
    AIMessage(content="4"), # AI 메시지: "4"
    HumanMessage(content="thanks"), # 사용자 메시지: "고마워"
    AIMessage(content="no problem!"), # AI 메시지: "천만에!"
    HumanMessage(content="having fun?"), # 사용자 메시지: "재미있어?"
    AIMessage(content="yes!"), # AI 메시지: "응!"
]

# trimmer를 사용하여 메시지 목록을 처리합니다.
# max_tokens=65 설정에 따라 메시지 목록이 줄어듭니다.
# 가장 최근 대화부터 65토큰을 유지하고, 시스템 메시지는 항상 포함합니다.
trimmed_messages = trimmer.invoke(messages)

# 잘려나간 메시지 목록을 출력합니다.
# [SystemMessage(content="you're a good assistant"),
#  HumanMessage(content='whats 2 + 2'),
#  AIMessage(content='4'),
#  HumanMessage(content='thanks'),
#  AIMessage(content='no problem!'),
#  HumanMessage(content='having fun?'),
#  AIMessage(content='yes!')]
# 'hi! I'm bob'과 'I like vanilla ice cream' 같은 초반 대화가 잘려나간 것을 볼 수 있습니다.

# API Reference:SystemMessage | trim_messages (관련 API 문서를 참고하세요)

# 체인(chain)에서 trim_messages를 사용하려면,
# 'messages' 입력을 프롬프트에 전달하기 전에 트리머를 실행하기만 하면 됩니다.

# 이제 모델에게 우리의 이름을 물어보면, 채팅 기록에서 해당 부분이 잘려나갔기 때문에 모델은 이름을 모를 것입니다.

from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
# RunnablePassthrough는 입력 값을 다음 단계로 그대로 전달하거나,
# 특정 키의 값을 다른 Runnable(실행 가능한 객체)로 처리하여 전달할 수 있게 해줍니다.

# 체인을 구성합니다.
chain = (
    # RunnablePassthrough.assign은 기존 입력에 새로운 키-값 쌍을 추가하거나 기존 값을 수정합니다.
    # 여기서는 "messages" 키의 값을 수정합니다.
    # itemgetter("messages")는 입력에서 "messages" 값을 가져옵니다.
    # 가져온 "messages" 값은 파이프(|)를 통해 trimmer로 전달되어 메시지가 잘려나갑니다.
    # 이렇게 잘려나간 메시지가 "messages" 키의 새로운 값이 됩니다.
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt # 프롬프트 템플릿에 메시지를 전달합니다. (여기서는 'prompt' 변수가 정의되지 않았지만, 실제 코드에서는 프롬프트 객체가 될 것입니다)
    | model # LLM 모델에 프롬프트와 메시지를 전달하여 응답을 생성합니다. (여기서도 'model' 변수가 정의되지 않았지만, 실제 코드에서는 LLM 모델 객체가 될 것입니다)
)

# 체인을 호출하여 응답을 받습니다.
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        # 기존 메시지 목록에 "내 이름이 뭐야?"라는 새로운 사용자 메시지를 추가합니다.
        "language": "English", # 언어 설정 (이 예제에서는 크게 중요하지 않습니다)
    }
)
response.content
# 결과: "I'm sorry, but I don't have access to your personal information. How can I assist you today?"
# 예상대로 모델은 이름을 기억하지 못합니다. "hi! I'm bob" 메시지가 잘려나갔기 때문입니다.

# API Reference:RunnablePassthrough (관련 API 문서를 참고하세요)

# 하지만 마지막 몇 개의 메시지 안에 있는 정보를 물어보면, 모델은 기억합니다.
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        # 기존 메시지 목록에 "내가 어떤 수학 문제를 물어봤지?"라는 새로운 사용자 메시지를 추가합니다.
        "language": "English",
    }
)
response.content
# 결과: 'You asked "what\'s 2 + 2?"'
# "whats 2 + 2" 메시지는 trimmer에 의해 잘려나가지 않고 남아있었기 때문에 모델이 기억합니다.

# 이제 이 체인을 'Message History'로 감싸보겠습니다.
# Message History는 이전 대화 기록을 저장하고 관리하는 기능을 제공합니다.
from langchain.memory import RunnableWithMessageHistory

# get_session_history 함수는 세션 ID에 따라 대화 기록을 불러오거나 저장하는 로직을 제공합니다.
# (이 예제에서는 get_session_history 함수가 정의되지 않았지만, 실제 코드에서는 구현되어야 합니다.)
def get_session_history(session_id: str):
    # 실제 애플리케이션에서는 이곳에 데이터베이스나 캐시에서 세션 기록을 로드하는 로직이 들어갑니다.
    # 예시를 위해 간단한 MemoryChatMessageHistory를 반환합니다.
    from langchain_community.chat_message_histories import ChatMessageHistory
    return ChatMessageHistory(session_id=session_id)

with_message_history = RunnableWithMessageHistory(
    chain, # 우리가 위에서 정의한 메시지 트리밍 체인입니다.
    get_session_history, # 세션 기록을 가져오는 함수입니다.
    input_messages_key="messages", # 체인의 입력에서 메시지 목록을 가져올 키입니다.
)

# 세션 ID를 설정하는 설정 객체입니다.
config = {"configurable": {"session_id": "abc20"}}

# 메시지 기록이 있는 체인을 호출합니다.
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        # 기존 메시지 목록에 "내 이름이 뭐야?"라는 새로운 사용자 메시지를 추가합니다.
        "language": "English",
    },
    config=config, # 세션 설정을 전달합니다.
)

response.content
# 결과: "I'm sorry, I don't have access to that information. How can I assist you today?"
# 예상대로, 우리가 이름을 말했던 첫 번째 메시지는 잘려나갔습니다.
# 게다가, 이제 채팅 기록에는 두 개의 새로운 메시지(우리의 최신 질문과 최신 응답)가 추가되었습니다.
# 이는 이전 대화 기록에서 접근 가능했던 정보가 더 이상 사용할 수 없게 되었음을 의미합니다!
# 이 경우, 우리의 초기 수학 문제도 기록에서 잘려나갔기 때문에 모델은 더 이상 그 내용을 알지 못합니다.

response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        # 새로운 대화 시작이므로, 새로운 메시지만 포함합니다.
        "language": "English",
    },
    config=config,
)

response.content
# 결과: "You haven't asked a math problem yet. Feel free to ask any math-related question you have, and I'll be happy to help you with it."
# "whats 2 + 2" 메시지가 이전에 잘려나갔거나, 새로운 대화 세션에서 시작되어 모델이 기억하지 못합니다.

# LangSmith를 살펴보면, 내부적으로 어떤 일이 일어나고 있는지 'LangSmith trace'에서 정확히 확인할 수 있습니다.
# LangSmith는 LangChain 애플리케이션의 실행 과정을 시각적으로 추적하고 디버깅하는 데 도움이 되는 도구입니다.

Couldn't find program: 'echo'


In [144]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [145]:
trim_messages = trimmer.invoke(messages)

In [147]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)
response.content

"I'm sorry, but I don't have access to personal information like your name unless you've shared it with me in this conversation. If you'd like, you can tell me your name!"

In [148]:
response.usage_metadata['total_tokens']

118

In [149]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked, "what\'s 2 + 2?"'

In [151]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc20"}}
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"I'm sorry, but I don't know your name. Could you please tell me if you would like me to address you by a specific name?"

In [150]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"I'm sorry, but I cannot recall past interactions or access previous questions you've asked. Could you please provide more details or restate the math problem you are referring to?"

In [ ]:
# 현재까지 설정된 환경 변수, 모델, 메시지 기록 함수 등은 이전 코드와 동일하게 유지됩니다.
# 편의를 위해 다시 한번 핵심 모듈 임포트와 변수 정의를 포함합니다.
import getpass
import os
from langchain_openai import ChatOpenAI
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

# API 키 설정 (이전과 동일)
os.environ["OPENAI_API_KEY"] = getpass.getpass()

# 모델 인스턴스 생성 (이전과 동일)
model = ChatOpenAI(model="gpt-3.5-turbo")

# 메시지 기록 저장소 및 함수 (이전과 동일)
store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# 언어 변수가 포함된 프롬프트 템플릿 (이전과 동일)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

# 프롬프트와 모델을 연결한 기본 체인 (이전과 동일)
chain = prompt | model

# 메시지 기록 기능을 포함한 체인 (이전과 동일)
# input_messages_key="messages"를 명시하여 메시지 기록을 어떤 키에서 가져올지 지정합니다.
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

# --- 스트리밍을 통한 사용자 경험 개선 ---

# "Now we've got a functioning chatbot."
# 이제 우리는 작동하는 챗봇을 만들었습니다.

# "However, one **really** important UX consideration for chatbot applications is streaming."
# 하지만 챗봇 애플리케이션에서 **정말** 중요한 사용자 경험(UX) 고려 사항 중 하나는 **스트리밍(streaming)**입니다.

# "LLMs can sometimes take a while to respond, and so in order to improve the user experience one thing that most applications do is stream back each token as it is generated. This allows the user to see progress."
# LLM은 때때로 응답하는 데 시간이 걸릴 수 있습니다. 따라서 사용자 경험을 개선하기 위해 대부분의 애플리케이션이 하는 한 가지는 각 토큰이 생성될 때마다 스트리밍하여 다시 보내는 것입니다. 이는 사용자가 진행 상황을 볼 수 있도록 합니다.

# "It's actually super easy to do this!"
# 사실 이건 매우 쉽습니다!

# "All chains expose a `.stream` method, and ones that use message history are no different. We can simply use that method to get back a streaming response."
# 모든 체인은 `.stream` 메서드를 노출하며, 메시지 기록을 사용하는 체인도 다르지 않습니다. 단순히 이 메서드를 사용하여 스트리밍 응답을 받을 수 있습니다.

# 사용할 세션 ID를 포함하는 config를 정의합니다.
config = {"configurable": {"session_id": "abc15"}}

# `with_message_history` 체인의 `.stream()` 메서드를 호출합니다.
# `.stream()` 메서드는 응답이 생성되는 동안 각 청크(부분)를 이터레이터로 반환합니다.
# 즉, 응답 전체가 완성될 때까지 기다리지 않고, 부분적으로 생성되는 토큰들을 즉시 받을 수 있습니다.
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="hi! I'm todd. tell me a joke")], # 현재 질문을 포함한 메시지
        "language": "English", # 원하는 응답 언어
    },
    config=config, # 메시지 기록을 위한 세션 config
):
    # 각 청크(r)는 LangChain의 BaseMessage 유형 (주로 AIMessage) 객체입니다.
    # r.content는 해당 청크의 텍스트 내용입니다.
    # `end="|"`는 각 토큰(또는 청크)이 출력될 때마다 줄 바꿈 대신 '|' 문자를 추가하여
    # 스트리밍되는 과정을 시각적으로 보여줍니다. (실제 챗봇에서는 단순히 print(r.content, end='')를 사용합니다.)
    print(r.content, end="|")

# 이 코드를 실행하면 모델이 농담을 생성하는 동안 각 단어(토큰)가 하나씩 출력되고 그 뒤에 '|'가 붙는 것을 볼 수 있습니다.
# 예를 들어, "Why|did|the|scarecrow|win|an|award|?|Because|he|was|outstanding|in|his|field!|" 와 같이 출력될 수 있습니다.

# 이 스트리밍 기능은 사용자가 모델의 응답을 기다리는 동안 지루함을 느끼지 않고,
# 응답이 생성되는 것을 실시간으로 확인할 수 있게 하여 챗봇의 사용자 경험을 크게 향상시킵니다.

In [ ]:
config = {"configurable": {"session_id": "abc15"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="hi! I'm todd. tell me a joke")],
        "language": "English",
    },
    config=config,
):
    print(r.content, end="|")